In [1]:
#import ppa
import ppa
import numpy as np
import matplotlib.pyplot as plt
from PTMCMCSampler.PTMCMCSampler import PTSampler
from chainconsumer import ChainConsumer
from num2tex import num2tex
import json
import scipy.stats as ss
    
PSR_DICT_LIST = ppa.Load_All_Pulsar_Info()
PSR_NAME_LIST = list(PSR_DICT_LIST.keys())
PSR_DICT_LIST

Do not have mpi4py package.
Do not have acor package


{'J0437-4715': {'PSR': 'J0437-4715',
  'RAJ': '04:37:15.8961737',
  'DECJ': '-47:15:09.110714',
  'DTE_DM': 0.156,
  'PX': 6.37,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0437-4715_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0437-4715_20cm.txt'},
  'RM': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0437-4715_ionFR_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0437-4715_ionFR_20cm.txt'}},
 'J0613-0200': {'PSR': 'J0613-0200',
  'RAJ': '06:13:43.975901',
  'DECJ': '-02:00:47.23742',
  'DTE_DM': 1.024,
  'PX': 1.01,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0613-0200_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0613-0200_20cm.txt'},
  'RM': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0613-0200_ionFR_10cm.txt',
   '20cm': 

In [10]:
#=====================================================#
#     M + EFAC + EQUAD                                #
#=====================================================#
Res = {}
for i,psrn in enumerate(PSR_NAME_LIST):  
    psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ],order=2,iono="subt" )
    res = {}
    for ss in ["10cm"]:#psr.SUBSETS:
        try:
            lnlike,lnprior,init_gen= psr.gen_spa_likelihood(ss,p=[-1,3,-8,2  ])
            
            # sampler = PTSampler(2,lnlike,lnprior,cov = np.diag(np.ones(2))*0.01,resume=False,outDir="SPA_Chain/m1_"+psr.PSR_NAME+"_"+ss,verbose=False)
            # sampler.sample(init_gen(),100000)
            
            chain  = np.loadtxt("SPA_Chain/m1_"+psrn+"_"+ss+"/chain_1.txt",skiprows=5000)


            med1 = np.median( chain[:,0])
            min1 = med1 - np.quantile( chain[:,0],0.16 )
            max1 = np.quantile( chain[:,0],0.84 ) - med1 


            med2 = np.median( chain[:,1])
            min2 = med2 - np.quantile( chain[:,1],0.16 )
            max2 = np.quantile( chain[:,1],0.84 ) - med2


            res.update( { ss:(med1,med2)} )
            logVol = np.log( 4 * 10 )
            logz0 = lnlike([0,-np.inf ])
            logBF = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0


            print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR[0])) ,"& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),"&&& %.1f"%(logBF)+"\\\\" )
        except:
            pass
    Res.update({psrn:res})

1 & J0437-4715 &-2.62 & $0.68^{+0.03}_{-0.03}$  & $-1.77^{+0.02}_{-0.02}$ &&& 43187.0\\
2 & J0613-0200 &-0.84 & $-0.19^{+0.05}_{-0.06}$  & $-1.02^{+0.05}_{-0.06}$ &&& 17.1\\
3 & J0614-3329 &-1.12 & $-0.13^{+0.06}_{-0.09}$  & $-2.61^{+1.06}_{-3.60}$ &&& 1.7\\
4 & J0711-6830 &-0.82 & $-0.13^{+0.02}_{-0.02}$  & $-1.23^{+0.05}_{-0.05}$ &&& 27.8\\
5 & J1017-7156 &-1.21 & $-0.11^{+0.05}_{-0.05}$  & $-1.26^{+0.04}_{-0.04}$ &&& 179.1\\
6 & J1022+1001 &-1.80 & $-0.00^{+0.03}_{-0.03}$  & $-2.00^{+0.06}_{-0.08}$ &&& 27.1\\
7 & J1024-0719 &-1.47 & $-0.02^{+0.03}_{-0.06}$  & $-2.13^{+0.34}_{-3.78}$ &&& -0.5\\
8 & J1045-4509 &-1.36 & $-0.04^{+0.02}_{-0.02}$  & $-4.95^{+2.11}_{-2.02}$ &&& 0.9\\
9 & J1125-6014 &-1.29 & $-0.16^{+0.02}_{-0.02}$  & $-5.00^{+2.05}_{-2.00}$ &&& 21.1\\
11 & J1545-4550 &-1.75 & $-0.18^{+0.06}_{-0.06}$  & $-1.78^{+0.04}_{-0.04}$ &&& 38.6\\
12 & J1600-3053 &-1.76 & $-0.03^{+0.05}_{-0.05}$  & $-1.84^{+0.05}_{-0.07}$ &&& 38.1\\
13 & J1603-7202 &-1.06 & $-0.11^{+0.02}_{-0.02}$  &

In [4]:
with open("ppa/Parfile/spa_results.json",'w') as f:
    spa_results = json.dump(Res,f,indent=2)

In [22]:
Res = {}
#=====================================================#
#    Summarize                                        #
#=====================================================#
test = 0
for i,psrn in enumerate(PSR_NAME_LIST[:]):  
    try:
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ] ,subset="10cm",iono="subt",order=2)
        lnlike,lnprior,init_gen = psr.gen_spa_likelihood("10cm")
        
        chain1  = np.loadtxt("SPA_Chain/m1_"+psrn+"_10cm/chain_1.txt",skiprows=5000)




        med1 = np.median( chain1[:,0])
        min1 = med1 - np.quantile( chain1[:,0],0.16 )
        max1 = np.quantile( chain1[:,0],0.84 ) - med1 


        med2 = np.median( chain1[:,1])
        min2 = med2 - np.quantile( chain1[:,1],0.16 )
        max2 = np.quantile( chain1[:,1],0.84 ) - med2 


        Res.update({psrn:{ "10cm":(med1,med2)}})
        
        logz0 = lnlike([0,-np.inf])
        #test += logz0

        logBF1 = np.log( 0.02 * 0.01 * 0.1 ) - np.log( np.mean(np.exp( -( chain1[:,-4]-chain1[:,-4].max() ) ) ) )  + chain1[:,-4].max() - logz0



        psr_2 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=2 )
        psr_1 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=1 )
        psr_0 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=0 )

        lnlike_2,lnprior,init_gen = psr_2.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        lnlike_1,lnprior,init_gen = psr_1.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        lnlike_0,lnprior,init_gen = psr_0.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        
        dlnlike10 = lnlike_1([0,-np.inf ]) - lnlike_0([0,-np.inf ])
        dlnlike21 =  lnlike_2([0,-np.inf ]) - lnlike_1([0,-np.inf ])



        # print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR)) ,
        #       "& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),
        #       "&%.1f&%.1f& %.1f"%(logBF1*np.log10(np.exp(1)),logBF2*np.log10(np.exp(1)),logBF*np.log10(np.exp(1)) )+"\\\\" )
        
        print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR)),"&%.2f"%np.log10(np.std(psr.DPA[0])) ,
            "  & $% .2f^{+%.2f}_{-%.2f}$& $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),
            "&${:.1f}$".format(num2tex(dlnlike10)) , "&${:.1f}$".format(num2tex(dlnlike21))  ,  "&${:.1f}$".format(num2tex(logBF1)) +"\\\\"  )
            #   "  &  %.1e  &  %.1e  &  %.1f "%(logBF1,logBF2 - logBF1,logBF3-logBF2 )+"\\\\" )
    except:
        pass

1 & J0437-4715 &-2.62 &-1.61   & $ 0.68^{+0.03}_{-0.03}$& $-1.77^{+0.02}_{-0.02}$ &$2177.8$ &$826.5$ &$43172.5$\\
2 & J0613-0200 &-0.84 &-0.83   & $-0.19^{+0.05}_{-0.06}$& $-1.02^{+0.05}_{-0.06}$ &$-1.2$ &$2.4$ &$2.6$\\
3 & J0614-3329 &-1.12 &-1.21   & $-0.13^{+0.06}_{-0.09}$& $-2.61^{+1.06}_{-3.60}$ &$-2.6$ &$-1.3$ &$-12.8$\\
4 & J0711-6830 &-0.82 &-0.78   & $-0.13^{+0.02}_{-0.02}$& $-1.23^{+0.05}_{-0.05}$ &$-3.0$ &$-1.6$ &$13.3$\\
5 & J1017-7156 &-1.21 &-1.09   & $-0.11^{+0.05}_{-0.05}$& $-1.26^{+0.04}_{-0.04}$ &$-3.8$ &$-0.1$ &$164.6$\\
6 & J1022+1001 &-1.80 &-1.60   & $-0.00^{+0.03}_{-0.03}$& $-2.00^{+0.06}_{-0.08}$ &$-1.3$ &$4.4$ &$12.6$\\
7 & J1024-0719 &-1.47 &-1.41   & $-0.02^{+0.03}_{-0.06}$& $-2.13^{+0.34}_{-3.78}$ &$-0.5$ &$-1.1$ &$-15.0$\\
8 & J1045-4509 &-1.36 &-1.34   & $-0.04^{+0.02}_{-0.02}$& $-4.95^{+2.11}_{-2.02}$ &$-0.6$ &$1.0$ &$-13.6$\\
9 & J1125-6014 &-1.29 &-1.30   & $-0.16^{+0.02}_{-0.02}$& $-5.00^{+2.05}_{-2.00}$ &$-3.7$ &$0.5$ &$6.6$\\
11 & J1545-4550 &-1.75 &

In [23]:
for i,psrn in enumerate(PSR_NAME_LIST[:]): 
    try:
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ] ,subset="10cm",iono="subt",order=2)
        print(psr.NOBS)
    except:
        pass

[1307]
[363]
[52]
[435]
[375]
[391]
[252]
[285]
[202]
[]
[228]
[344]
[341]
[297]
[381]
[260]
[]
[373]
[144]
[]
[212]
[1098]
[184]
[]
[]
[350]
[409]
